In [1]:
from notebook_utils import *

In [2]:
experiments_ids = [BitflipExperimentID.IPMA2, PhaseflipExperimentID.IPMA, ResetExperimentID.main]

In [7]:
mc_guarantees_file_name = "mc_guarantees_.csv"
counts_dicts = dict() # horizon -> List[hardware_specs]
all_lambdas = load_all_lambdas(ResetExperimentID.main)
for experiment_id in [ResetExperimentID.main]:
    file_path = os.path.join(get_project_path(), get_experiment_name_path(experiment_id), experiment_id.value, mc_guarantees_file_name)
    f = open(file_path)
    lines = f.readlines()[1:]
    f.close()
    for line in lines:
        elements = line.split(",")
        hardware = elements[0]
        embedding_index = int(elements[1])
        horizon = int(elements[2])
        my_guarantee = float(elements[3])
        trad_guarantee = float(elements[4])
        best_trad_guarantee = float(elements[6])
        if my_guarantee >= 0.999 and (best_trad_guarantee != my_guarantee):
            if horizon not in counts_dicts.keys():
                counts_dicts[horizon] = []
            counts_dicts[horizon].append((hardware, embedding_index, all_lambdas[find_enum_object(hardware, HardwareSpec)][embedding_index][horizon]))
counts_dicts

{6: [('fake_belem', 0, 0.9995763545600939),
  ('fake_london', 1, 0.9999292698943443),
  ('fake_quito', 0, 0.9998415520778541),
  ('fake_quito', 1, 0.9991861628175757),
  ('fake_yorktown', 2, 0.9995542267206531),
  ('fake_johannesburg', 3, 0.9997839672430161),
  ('fake_almaden', 3, 0.9998334763435842),
  ('fake_almaden', 5, 0.9998898400317076),
  ('fake_boeblingen', 1, 0.9998042970927153),
  ('fake_poughkeepsie', 0, 0.9997934808043892),
  ('fake_tokyo', 3, 0.9995641049032341),
  ('fake_tokyo', 5, 0.9997439706089121),
  ('fake_nairobi', 0, 0.9995639064706446),
  ('fake_nairobi', 2, 0.9995394192249738),
  ('fake_cairo', 6, 0.9999472507374284),
  ('fake_sydney', 0, 0.9994844904373222),
  ('fake_cambridge', 3, 0.9998603543796738),
  ('fake_melbourne', 4, 0.9992838536063355),
  ('fake_rochester', 3, 0.9999557723146539)],
 7: [('fake_burlington', 1, 0.9997620765054932),
  ('fake_essex', 0, 0.9998480686040007),
  ('fake_essex', 1, 0.9996630217301619),
  ('fake_lima', 1, 0.9995157418427825),
  

In [20]:
for (horizon, hardware_specs) in counts_dicts.items():
    print(horizon, len(hardware_specs))

7 14
6 7
5 9


In [34]:
experiment_id = GHZExperimentID.EMBED
file_path = os.path.join(get_project_path(), get_experiment_name_path(experiment_id),"guarantees_embed.csv")
f = open(file_path)
lines = f.readlines()[1:]
f.close()
for line in lines:
    elements = line.split(",")
    hardware = elements[0]
    my_guarantee = float(elements[1])
    trad_guarantee = float(elements[2])
    diff = float(elements[3])
    if my_guarantee >= 0.99 and (my_guarantee != trad_guarantee):
        print(hardware,my_guarantee, trad_guarantee, diff)



fake_hanoi 0.99 0.988 0.002
fake_auckland 0.99 0.979 0.011
fake_geneva 0.99 0.982 0.008
